In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import pandas as pd
import collections
from IPython.display import display, clear_output

file_path = "path-Sentiment_Analysis.csv"
try:
    data = pd.read_csv(file_path)
    print("Data loaded successfully. First 5 rows:")
    display(data.head())
except Exception as e:
    print(f"Error loading file: {e}")
    raise

print("\nAvailable columns:", data.columns.tolist())

column_mapping = {
    'text': ['text', 'sentence', 'review', 'tweet', 'comment', 'phrase', 'content'],
    'sentiment': ['sentiment', 'label', 'category', 'class', 'rating', 'emotion']
}

text_col = next((col for col in data.columns
                if any(kw in col.lower() for kw in column_mapping['text'])), None)

sentiment_col = next((col for col in data.columns
                     if any(kw in col.lower() for kw in column_mapping['sentiment'])), None)

if not text_col or not sentiment_col:
    text_col, sentiment_col = data.columns[:2]
    print(f"\n Using first two columns as: Text='{text_col}', Sentiment='{sentiment_col}'")
else:
    print(f"\n Auto-detected columns: Text='{text_col}', Sentiment='{sentiment_col}'")

print("\nBuilding sentiment graph...")
graph = collections.defaultdict(list)

sentiment_groups = data.groupby(sentiment_col)[text_col].apply(list)

for sentiment, sentences in sentiment_groups.items():
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            graph[sentences[i]].append(sentences[j])
            graph[sentences[j]].append(sentences[i])

print(f"Graph built with {len(graph)} nodes")

def bfs(graph, start_node):
    if start_node not in graph:
        raise ValueError(f"Start node not found in graph: '{start_node}'")

    visited = set()
    queue = collections.deque([start_node])
    visited.add(start_node)
    traversal_order = [start_node]

    while queue:
        current_node = queue.popleft()
        for neighbor in graph.get(current_node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                traversal_order.append(neighbor)

    return traversal_order

start_node = data[text_col].iloc[0]
print(f"\nStarting BFS with node: '{start_node[:50]}...'")

try:
    traversal_order = bfs(graph, start_node)
    print(f"\nBFS traversal completed. Visited {len(traversal_order)} nodes.")
    print("\nFirst 5 nodes in traversal order:")
    for i, node in enumerate(traversal_order[:5], 1):
        print(f"{i}. {node[:70]}...")
except ValueError as e:
    print(f"\nError: {e}")
    print("Available nodes sample:", list(graph.keys())[:3])
